In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm
import time
from IPython.display import clear_output

import ee, geemap

In [2]:
%load_ext autoreload
%autoreload 2
import sys  
sys.path.insert(0, '../../lib/')

In [91]:
%reload_ext autoreload
import sand_classification as sc
import quality_mosaic as qm

# Part I: Get the locations from Google Drive

In [4]:
sheet_id = '13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos'
# sheet_id = '1XqOtPkiE_Q0dfGSoyxrH730RkwrTczcRbDeJJpqRByQ'
sheet_name = 'mozambique'
# sheet_name = 'sample_1'
drive_url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
drive_url

'https://docs.google.com/spreadsheets/d/13nF_pJ02Bd70cDJamuKbvZIkIdJ-kOI4O3Cx9K7Wzos/gviz/tq?tqx=out:csv&sheet=mozambique'

In [5]:
#Read in a google sheets with the ground truth data from string drive_loc
df = pd.read_csv(drive_url)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [6]:
#look for any NaN values in the column 'date' or 'Longitude' or 'Latitude' in df_merged and drop them
df = df.dropna(subset=['Date', 'Longitude', 'Latitude']).reset_index(drop=True)
df

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [7]:
assert(len(df[df['Date'].isna() | df['Longitude'].isna() | df['Latitude'].isna()]) == 0)

# Part II: Select subset for training

Here we iterate over the df and iteratively pick and adjust the locations we want to keep

In [8]:
df.columns

Index(['ID', 'Latitude', 'Longitude', 'Class', 'Date', 'Site', 'Source',
       'Country', 'Notes'],
      dtype='object')

In [9]:
ee.Initialize()
# ee.Authenticate()

In [31]:
output = df.copy()
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19)
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01)
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31)
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61)


In [32]:
output = qm.setup_output_bands(df.copy())
output


,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2,...,B8A,B11,B12,VV,VH,mTGSI,BSI,NDWI,keep,location_tweaked
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False
5,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [85]:
INDEX = 0
sampling_buffer_m = 5
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

## Manual Loop

Keep running the following 2 cells until you're done with the manual sampling.


In [113]:
INDEX = 5

In [115]:
s1_s2, sample = qm.refresh_maps_one_at_a_time(output, Map, INDEX, display_smap=True, mosaic_method='qm', sampling_buffer_m=sampling_buffer_m, median_days=20)

Index:  5  ID:  moz-5 Class:  sand  Country:  Mozambique  Site:  fluvial
Search window from 2022-01-22 to 2023-01-22


Map(center=[-25.353036991916163, 32.32731207641525], controls=(ZoomControl(options=['position', 'zoom_in_text'…

## Use this immediately after the map is refreshed to either select or discard the sample

INDEX will auto update on calling this function

In [116]:
output, INDEX = qm.get_values_one_at_a_time(output, s1_s2, sample, Map, INDEX)
print(output[['Latitude', 'Longitude','Class','keep','location_tweaked']].loc[INDEX-1])

New marker accepted
Kept Observation
Latitude           -25.353037
Longitude           32.327312
Class                    sand
keep                     True
location_tweaked         True
Name: 5, dtype: object


In [117]:
INDEX

6

In [118]:
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2,...,B8A,B11,B12,VV,VH,mTGSI,BSI,NDWI,keep,location_tweaked
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1424.775801,...,3415.000000,4678.000000,4285.000000,-15.051572,-26.791706,0.162989,0.218167,-0.223185,True,True
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,1323.618705,...,3344.366906,4756.661871,4745.467626,-15.421135,-27.160017,0.204982,0.238582,-0.256787,True,False
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),1025.946237,...,3013.000000,4363.000000,4366.000000,-18.701325,-29.746097,0.219025,0.251916,-0.294739,True,False
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),986.097826,...,2851.134058,4177.449275,4213.413043,-18.801203,-33.041720,0.224025,0.254158,-0.304060,True,False
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),1508.221429,...,3616.185714,5096.050000,4711.221429,-14.737598,-25.256125,0.171833,0.225222,-0.233366,True,False
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),1336.747331,...,3605.000000,5036.000000,4777.000000,-16.226424,-24.205281,0.180526,0.225067,-0.293994,True,True
6,moz-5,-25.353037,32.327312,sand,2016-12-23,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False


In [119]:
#delete the last row of output
output = output.drop(output.index[INDEX])
output

,ID,Latitude,Longitude,Class,Date,Site,Source,Country,Notes,B2,...,B8A,B11,B12,VV,VH,mTGSI,BSI,NDWI,keep,location_tweaked
0,moz-0,-25.571043,32.252410,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10747CM applied for,1424.775801,...,3415.000000,4678.000000,4285.000000,-15.051572,-26.791706,0.162989,0.218167,-0.223185,True,True
1,moz-1,-24.505472,33.012067,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 10128CM Granted (2020-02-03 to 2030-02...,1323.618705,...,3344.366906,4756.661871,4745.467626,-15.421135,-27.160017,0.204982,0.238582,-0.256787,True,False
2,moz-2,-24.647689,33.283443,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 6528CM Granted (2013-08-22 to 2026-01-19),1025.946237,...,3013.000000,4363.000000,4366.000000,-18.701325,-29.746097,0.219025,0.251916,-0.294739,True,False
3,moz-3,-24.633682,33.283166,sand,2019-08-15,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8996CM Granted (2018-04-01 to 2028-04-01),986.097826,...,2851.134058,4177.449275,4213.413043,-18.801203,-33.041720,0.224025,0.254158,-0.304060,True,False
4,moz-4,-25.563075,32.209854,sand,2021-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 3507CM Granted (2015-03-31 to 2017-03-31),1508.221429,...,3616.185714,5096.050000,4711.221429,-14.737598,-25.256125,0.171833,0.225222,-0.233366,True,False
5,moz-5,-25.353037,32.327312,sand,2022-07-22,fluvial,https://portals.landfolio.com/mozambique/en/,Mozambique,License 8348C Applied (2016-09-61),1336.747331,...,3605.000000,5036.000000,4777.000000,-16.226424,-24.205281,0.180526,0.225067,-0.293994,True,True


In [120]:
output.to_csv('../../data/mozambique-gt-final-locations.csv')
#save fm_df to a pickle file
output.to_pickle('../../data/mozambique-gt-final-locations.pkl')